In [43]:
import pandas as pd
from lifelines import CoxPHFitter
from src.helper_methods import *
from src.pipe_store import *
from src.constants import *
from lifelines import CoxPHFitter

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Loading
### Deceased Donors

In [44]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'
donor_type ='Deceased'
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']

df = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(polynomial_power2, 'DonorAge_NOTR', 'RecipientAge_NOTR')
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    .pipe(eng_immunological_features, antibody_epitope=True)
    .pipe(feature_scaler, num_col + ['DonorAge_NOTR_power2', 'RecipientAge_NOTR_power2'], scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)
df = create_treatment_grups(df, [RELEVANT_DESA_BAD])

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.013642s
- selceted cohort --> Donor Type: Deceased, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (3235, 23) | Computation Time: 0.007375s
Step: integer_encoder | Shape: (3235, 12) | Computation Time: 0.000975s
Step: set_time_event_label | Shape: (3235, 14) | Computation Time: 0.002321s
Step: eng_immunological_features | Shape: (3235, 19) | Computation Time: 0.027217s
Step: censoring_deaths | Shape: (3235, 19) | Computation Time: 0.001279s
Step: setting_prediction_horizon | Shape: (3235, 19) | Computation Time: 0.003509s


# Structural Causal Models

# Checking the Cox Proportional Hazard Assumptions

https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html?highlight=cox%20assumptions

The proportional hazard assumption is that all individuals have the same hazard function, but a unique scaling factor infront. So the shape of the hazard function is the same for all individuals, and only a scalar multiple changes per individual.

# Deceased Donors

In [45]:
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR_power2',
    'DonorAge_NOTR',
    'DonorAge_NOTR_power2',
    'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
    'DialysisYears',
    # 'No_DESA', 
    'Specific_DESA', 
    # 'Other_DESA'
]

In [46]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] >= 1]
df_early = df_total[df_total['T'] < 1]
df_class_I = df_total[df_total['Class_I'] == 1]
df_class_II = df_total[df_total['Class_II'] == 1]
df_class_I_II = df_total[df_total['Class_I_II'] == 1]

formula = ' + '.join(confounders)
# # for df_l, title in [(df_total, 'Total Data Set'), (df_late, 'Late Failure'), (df_early, 'Early Failure')]:
df_l = df_total
cph = CoxPHFitter()
cph.fit(df_l, duration_col='T', event_col='E', formula=formula)
len = df_l['Specific_DESA'].sum()
print(f' Bad DESA size in data set {len}')
cph.print_summary()

 Bad DESA size in data set 148


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 3235 total observations, 2324 right-censored observations>
             duration col = 'T'
                event col = 'E'
      baseline estimation = breslow
   number of observations = 3235
number of events observed = 911
   partial log-likelihood = -6908.87
         time fit was run = 2022-06-28 12:18:05 UTC

---
                           coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                        
CIPHour_DBD                0.02       1.03       0.06            -0.08             0.13                 0.92                 1.14
CIPHour_DCD                0.22       1.24       0.05             0.12             0.32                 1.12                 1.37
DialysisYears              0.00       1.00       0.03            -0.06             0.07                 0.94                 1.07
DonorAge_NOTR             -0.39       0.68       0.16            -0.71            -0.07                 0.49                 0.93
DonorAge_NOTR_power2       0.75       2.11       0.16             0.44             1.05                 1.56                 2.87
IL2rMoAb                   0.24       1.27       0.09             0.06             0.42                 1.06                 1.52
RecipientAge_NOTR         -0.31       0.74       0.16            -0.61            -0.00                 0.54                 1.00
RecipientAge_NOTR_power2   0.09       1.10       0.16            -0.23             0.41                 0.80                 1.51
Specific_DESA              1.10       3.01       0.11             0.88             1.32                 2.42                 3.74

                           cmp to     z      p   -log2(p)
covariate                                                
CIPHour_DBD                  0.00  0.45   0.66       0.61
CIPHour_DCD                  0.00  4.22 <0.005      15.30
DialysisYears                0.00  0.15   0.88       0.18
DonorAge_NOTR                0.00 -2.41   0.02       5.96
DonorAge_NOTR_power2         0.00  4.79 <0.005      19.20
IL2rMoAb                     0.00  2.63   0.01       6.88
RecipientAge_NOTR            0.00 -1.96   0.05       4.33
RecipientAge_NOTR_power2     0.00  0.58   0.56       0.83
Specific_DESA                0.00  9.92 <0.005      74.63
---
Concordance = 0.66
Partial AIC = 13835.74
log-likelihood ratio test = 277.87 on 9 df
-log2(p) of ll-ratio test = 179.03

|Deceased donors (3235)| #Tx with Bad DESA | Bad DESA (HR) | 95% CI|
| ------------- | --------------| -----------|-----------|
| All           |         148|      3.02  | 2.43-3.75
| DESA Class I  |    73      |      3.11 |  1.91-5.04
| DESA Class II |      51    |    2.27   |  1.33-3.85
| DESA Class I & II| 24      |     1.39| 0.33-5.94	
| Early failures (< 1 y)| 41 |  1.63  | 1.15-2.32
| Late failures (>1 y)| 107  |    3.23|2.44-4.27

# Living Donor

In [47]:
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR_power2',
    'DonorAge_NOTR',
    'DonorAge_NOTR_power2',
    'IL2rMoAb', 
    # 'CIPHour_DBD', 'CIPHour_DCD', 
    # 'No_DESA', 
    'Specific_DESA', 
    'DialysisYears',
    # 'Other_DESA'
]

In [52]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'

donor_type ='Living'
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR']

df = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(polynomial_power2, 'DonorAge_NOTR', 'RecipientAge_NOTR')
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    .pipe(eng_immunological_features, antibody_epitope=True)
    .pipe(feature_scaler, num_col + ['DonorAge_NOTR_power2', 'RecipientAge_NOTR_power2'], scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)
df = create_treatment_grups(df, [RELEVANT_DESA_BAD])


Step: data_loading | Shape: (4690, 24) | Computation Time: 0.022823s
Step: data_loading | Shape: (4690, 24) | Computation Time: 0.022823s
- selceted cohort --> Donor Type: Living, Epitope Antibody Presence: All
- selceted cohort --> Donor Type: Living, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (1455, 23) | Computation Time: 0.011752s
Step: start_pipeline | Shape: (1455, 23) | Computation Time: 0.011752s
Step: integer_encoder | Shape: (1455, 10) | Computation Time: 0.000634s
Step: integer_encoder | Shape: (1455, 10) | Computation Time: 0.000634s
Step: set_time_event_label | Shape: (1455, 12) | Computation Time: 0.001354s
Step: set_time_event_label | Shape: (1455, 12) | Computation Time: 0.001354s
Step: eng_immunological_features | Shape: (1455, 17) | Computation Time: 0.014152s
Step: eng_immunological_features | Shape: (1455, 17) | Computation Time: 0.014152s
Step: censoring_deaths | Shape: (1455, 17) | Computation Time: 0.00157s
Step: censoring_deaths | Shape: (1455,

In [53]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] >= 1]
df_early = df_total[df_total['T'] < 1]
df_class_I = df_total[df_total['Class_I'] == 1]
df_class_II = df_total[df_total['Class_II'] == 1]
df_class_I_II = df_total[df_total['Class_I_II'] == 1]

formula = ' + '.join(confounders)
# for df_l, title in [(df_total, 'Total Data Set'), (df_late, 'Late Failure'), (df_early, 'Early Failure')]:
df_l = df_class_I_II
cph = CoxPHFitter()
cph.fit(df_l, duration_col='T', event_col='E', formula=formula)
len = df_l['Specific_DESA'].sum()
print(f' Bad DESA size in data set {len}')
cph.print_summary()

 Bad DESA size in data set 9


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/__init__.py:1123: ConvergenceWarning: Column IL2rMoAb have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['E'].astype(bool)
>>> print(df.loc[events, 'IL2rMoAb'].var())
>>> print(df.loc[~events, 'IL2rMoAb'].var())

A very low variance means that the column IL2rMoAb completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1601: ConvergenceWarning: Newton-Rhaphson convergence completed successfully but norm(delta) is still high, 0.332. This may imply non-unique

<lifelines.CoxPHFitter: fitted with 17 total observations, 12 right-censored observations>
             duration col = 'T'
                event col = 'E'
      baseline estimation = breslow
   number of observations = 17
number of events observed = 5
   partial log-likelihood = -10.57
         time fit was run = 2022-06-28 12:28:43 UTC

---
                           coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                        
DialysisYears             -0.01       0.99       0.60            -1.19             1.17                 0.30                 3.24
DonorAge_NOTR              5.66     287.57       7.12            -8.30            19.62                 0.00             3.31e+08
DonorAge_NOTR_power2      -6.41       0.00       8.40           -22.87            10.06                 0.00             23277.21
IL2rMoAb                 -16.40       0.00    3130.79         -6152.64          6119.85                 0.00                  inf
RecipientAge_NOTR         -0.79       0.46       5.08           -10.73             9.16                 0.00              9522.31
RecipientAge_NOTR_power2   0.61       1.85       5.46           -10.09            11.32                 0.00             82728.09
Specific_DESA              1.38       3.99       1.49            -1.54             4.31                 0.21                74.09

                           cmp to     z    p   -log2(p)
covariate                                              
DialysisYears                0.00 -0.02 0.99       0.02
DonorAge_NOTR                0.00  0.80 0.43       1.23
DonorAge_NOTR_power2         0.00 -0.76 0.45       1.17
IL2rMoAb                     0.00 -0.01 1.00       0.01
RecipientAge_NOTR            0.00 -0.15 0.88       0.19
RecipientAge_NOTR_power2     0.00  0.11 0.91       0.14
Specific_DESA                0.00  0.93 0.35       1.50
---
Concordance = 0.75
Partial AIC = 35.14
log-likelihood ratio test = 3.33 on 7 df
-log2(p) of ll-ratio test = 0.23

|Living donors 1455| #Tx with Bad DESA | Bad DESA (HR) | 95% CI|
| ------------- | --------------| -----------|-----------|
| All           |        48 |       1.66      | 0.97-2.85	
| DESA Class I  | 16         |      2.75      | 0.99-7.61		
| DESA Class II | 23         |   1.01	        | 0.02-1.26	
| DESA Class I & II| 9      |       4.03	     | 0.28-58.10	
| Early failures (< 1 y)| 5  |      3.57   | 1.07-11.87
| Late failures (>1 y)| 43      |  1.43	 | 0.76-2.71

# Potential Outcomes

## Finding Inverse Probability Wieghts (IPW)

In [10]:
df

,EpvsHLA_Donor,#DESA,DESA,DonorAge_NOTR,DialysisYears,RecipientAge_NOTR,CIPHour_DBD,CIPHour_DCD,Donor_Type,IL2rMoAb,DonorAge_NOTR_power2,RecipientAge_NOTR_power2,E,T,No_DESA,Specific_DESA,Other_DESA,Groups
0,{},0,{},0.078804,-0.431079,0.506022,-0.645746,-0.545901,Deceased,0,-0.113195,0.414553,0,12.191781,1,0,0,1
1,{},0,{},0.704858,-0.509609,-1.059379,0.466621,-0.545901,Deceased,0,0.647260,-1.093706,0,15.000000,1,0,0,1
2,{},0,{},0.078804,0.589814,-1.415152,1.709855,-0.545901,Deceased,0,-0.113195,-1.328873,1,6.956164,1,0,0,1
3,{},0,{},-1.235912,-1.334176,-0.490142,1.186388,-0.545901,Deceased,1,-1.204992,-0.634532,0,4.509589,1,0,0,1
4,{},0,{},0.767464,0.786139,-0.632451,-1.430947,1.175755,Deceased,0,0.731841,-0.758892,1,0.010959,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4685,{},0,{},0.517042,-0.745199,-0.276678,1.099144,-0.545901,Deceased,1,0.402828,-0.436035,0,12.126027,1,0,0,1
4686,{},0,{},0.704858,-0.666669,-0.703606,0.375014,-0.545901,Deceased,1,0.647260,-0.818680,1,10.879452,1,0,0,1
4687,{},0,{},-1.361123,0.040103,-0.063214,0.068204,-0.545901,Deceased,1,-1.273278,-0.223189,0,15.000000,1,0,0,1
4688,{},0,{},-0.046407,-0.352548,-1.699770,-1.430947,2.672412,Deceased,1,-0.246663,-1.488308,1,11.816438,1,0,0,1


In [11]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] > 1]
df_early = df_total[df_total['T'] < 1]
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR_power2',
    'DonorAge_NOTR',
    'DonorAge_NOTR_power2',
    'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
]
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
df_weight = find_ipw(df, confounders, treatments, scaler=None, verbose=False)

cph = CoxPHFitter()
cols = [
    # 'No_DESA',
    'Specific_DESA',
    'Other_DESA',
    'E', 'T', 
    'w'
]
cph.fit(
    df_weight[cols], 
    weights_col='w',
    duration_col='T', event_col='E', robust=True)
print(cph.print_summary())

['No_DESA ~ RecipientAge_NOTR + RecipientAge_NOTR_power2 + DonorAge_NOTR + DonorAge_NOTR_power2 + IL2rMoAb + CIPHour_DBD + CIPHour_DCD', 'Other_DESA ~ RecipientAge_NOTR + RecipientAge_NOTR_power2 + DonorAge_NOTR + DonorAge_NOTR_power2 + IL2rMoAb + CIPHour_DBD + CIPHour_DCD', 'Specific_DESA ~ RecipientAge_NOTR + RecipientAge_NOTR_power2 + DonorAge_NOTR + DonorAge_NOTR_power2 + IL2rMoAb + CIPHour_DBD + CIPHour_DCD']


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 9678.44 total observations, 5861.51 right-censored observations>
             duration col = 'T'
                event col = 'E'
              weights col = 'w'
          robust variance = True
      baseline estimation = breslow
   number of observations = 9678.44
number of events observed = 3816.93
   partial log-likelihood = -32960.71
         time fit was run = 2022-04-07 07:37:19 UTC

---
                coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                             
Specific_DESA   1.16       3.19       0.12             0.92             1.40                 2.51                 4.05
Other_DESA      0.24       1.27       0.14            -0.03             0.51                 0.97                 1.67

                cmp to    z      p   -log2(p)
covariate                                    
Specific_DESA     0.00 9.49 <0.005      68.57
Other_DESA        0.00 1.74   0.08       3.60
---
Concordance = 0.54
Partial AIC = 65925.42
log-likelihood ratio test = 999.02 on 2 df
-log2(p) of ll-ratio test = 720.64

None


In [39]:
cph.check_assumptions(df_weight[cols], p_value_threshold=0.05, show_plots=True)

Proportional hazard assumption looks okay.


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/kaplan_meier_fitter.py:317: StatisticalWarning: It looks like your weights are not integers, possibly propensity scores then?
  It's important to know that the naive variance estimates of the coefficients are biased. Instead use Monte Carlo to
  estimate the variances. See paper "Variance estimation when using inverse probability of treatment weighting (IPTW) with survival analysis"
  or "Adjusted Kaplan-Meier estimator and log-rank test with inverse probability of treatment weighting for survival data."
                  
  warnings.warn(
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/mixins.py:108: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  for variable in self.params_.index & (colu

[]

In [40]:
from lifelines.statistics import proportional_hazard_test
results = proportional_hazard_test(cph, df_weight[cols], time_transform='rank')
results.print_summary(decimals=3, model="untransformed variables")

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/statistics.py:143: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return self.summary.to_latex()


<lifelines.StatisticalResult: proportional_hazard_test>
    time_transform = rank
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 9688.73 total observations, 5829.95 right-censored observations>
         test_name = proportional_hazard_test

---
               test_statistic    p  -log2(p)
Other_DESA               0.03 0.87      0.20
Specific_DESA            0.12 0.73      0.45

### Living Donor

In [52]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'
donor_type ='Living'
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']

df = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        # 'TypeCadaveric_NOTR', 
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(features_from_antibody_epitopes)
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    # .pipe(feature_scaler, num_col, scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)
df['DESA_3more'] = df['#DESA'].apply(lambda x: 1 if x > 2 else 0)

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.01847s
- selceted cohort --> Donor Type: Living, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (1455, 23) | Computation Time: 0.013016s
Step: integer_encoder | Shape: (1455, 12) | Computation Time: 0.000888s
Step: set_time_event_label | Shape: (1455, 16) | Computation Time: 0.004486s
Step: censoring_deaths | Shape: (1455, 16) | Computation Time: 0.002584s
Step: setting_prediction_horizon | Shape: (1455, 16) | Computation Time: 0.005771s


In [53]:
df.sample()

,#DESA,DESA,DonorAge_NOTR,DialysisYears,RecipientAge_NOTR,CIPHour_DBD,CIPHour_DCD,Donor_Type,IL2rMoAb,No_DESA,Relevant_DESA_Bad,Relevant_DESA_Good,Class_I,Class_II,E,T,DESA_3more
838,0,{},20,2.7,44,0.0,0.0,Living,0,1,0,0,0,0,0,15.0,0


In [16]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] > 1]
df_early = df_total[df_total['T'] < 1]

# for df, title in [(df_total, 'Total Data Set'), (df_late, 'Late Failure'), (df_early, 'Early Failure')]:
df = df_total
confounders = [
    'RecipientAge_NOTR', 'RecipientAge_NOTR*RecipientAge_NOTR', 
    'DonorAge_NOTR', 'DonorAge_NOTR*DonorAge_NOTR', 'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
]

df_treat_group = create_treatment_grups(df, [RELEVANT_DESA_BAD])
# treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA', 'DESA_3more']
# df_weight = find_ipw(df_treat_group, confounders, treatments, verbose=False)
df_treat_group
# cph = CoxPHFitter()
# cols = [
#     'DonorAge_NOTR', 'RecipientAge_NOTR',
#     'DialysisYears', 'IL2rMoAb',
#     'CIPHour_DBD', 'CIPHour_DCD', 
#     'No_DESA',
#     'Specific_DESA',
#     'DESA_3more',
#     'E', 'T', 'w',
# ]
# cph.fit(df_weight[cols], weights_col='w', duration_col='T', event_col='E', robust=True)
# # print(title)
# cph.print_summary()


,#DESA,DESA,DonorAge_NOTR,DialysisYears,RecipientAge_NOTR,CIPHour_DBD,CIPHour_DCD,TypeCadaveric_NOTR,Donor_Type,IL2rMoAb,...,Relevant_DESA_Bad,Relevant_DESA_Good,Class_I,Class_II,E,T,DESA_3more,Specific_DESA,Other_DESA,Groups
0,0,{},44,2.3,54,9.000000,0.000000,0,Deceased,0,...,0,0,0,0,0,12.191781,0,0,0,1
1,0,{},54,2.1,32,21.750000,0.000000,0,Deceased,0,...,0,0,0,0,0,15.000000,0,0,0,1
2,0,{},44,4.9,27,36.000000,0.000000,0,Deceased,0,...,0,0,0,0,1,6.956164,0,0,0,1
3,0,{},23,0.0,40,30.000000,0.000000,0,Deceased,1,...,0,0,0,0,0,4.509589,0,0,0,1
4,0,{},55,5.4,38,0.000000,17.216667,1,Deceased,0,...,0,0,0,0,1,0.010959,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4685,0,{},51,1.5,43,29.000000,0.000000,0,Deceased,1,...,0,0,0,0,0,12.126027,0,0,0,1
4686,0,{},54,1.7,37,20.700000,0.000000,0,Deceased,1,...,0,0,0,0,1,10.879452,0,0,0,1
4687,0,{},21,3.5,46,17.183333,0.000000,0,Deceased,1,...,0,0,0,0,0,15.000000,0,0,0,1
4688,0,{},42,2.5,23,0.000000,32.183333,1,Deceased,1,...,0,0,0,0,1,11.816438,0,0,0,1


# Total Data

In [39]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'
donor_type = None
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']

df = (
    data_loading(path)
    # .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(polynomial_power2, 'DonorAge_NOTR', 'RecipientAge_NOTR')
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    .pipe(eng_immunological_features, antibody_epitope=True)
    .pipe(feature_scaler, num_col + ['DonorAge_NOTR_power2', 'RecipientAge_NOTR_power2'], scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 10)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)
df = create_treatment_grups(df, [RELEVANT_DESA_BAD])

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.013059s
Step: integer_encoder | Shape: (4690, 12) | Computation Time: 0.001571s
Step: set_time_event_label | Shape: (4690, 14) | Computation Time: 0.001867s
Step: eng_immunological_features | Shape: (4690, 19) | Computation Time: 0.038885s
Step: censoring_deaths | Shape: (4690, 19) | Computation Time: 0.001222s
Step: setting_prediction_horizon | Shape: (4690, 19) | Computation Time: 0.004254s


In [40]:
df

,#DESA,DESA,DonorAge_NOTR,DialysisYears,RecipientAge_NOTR,CIPHour_DBD,CIPHour_DCD,Donor_Type,IL2rMoAb,DonorAge_NOTR_power2,...,T,No_DESA,Relevant_DESA_Bad,Relevant_DESA_Good,Class_I,Class_II,Class_I_II,Specific_DESA,Other_DESA,Groups
0,0,{},-0.019920,-0.183046,0.596332,-0.190025,-0.433789,Deceased,0,-0.200724,...,10.000000,1,0,0,0,0,0,0,0,1
1,0,{},0.645513,-0.263943,-0.936160,0.857348,-0.433789,Deceased,0,0.579189,...,10.000000,1,0,0,0,0,0,0,0,1
2,0,{},-0.019920,0.868621,-1.284454,2.027942,-0.433789,Deceased,0,-0.200724,...,6.956164,1,0,0,0,0,0,0,0,1
3,0,{},-1.417332,-1.113366,-0.378890,1.535061,-0.433789,Deceased,1,-1.320455,...,4.509589,1,0,0,0,0,0,0,0,1
4,0,{},0.712057,1.070865,-0.518208,-0.929348,1.549608,Deceased,0,0.665934,...,0.010959,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4685,0,{},0.445883,-0.506635,-0.169914,1.452914,-0.433789,Deceased,1,0.328503,...,10.000000,1,0,0,0,0,0,0,0,1
4686,0,{},0.645513,-0.425738,-0.587867,0.771094,-0.433789,Deceased,1,0.579189,...,10.000000,1,0,0,0,0,0,0,0,1
4687,0,{},-1.550418,0.302339,0.039062,0.482210,-0.433789,Deceased,1,-1.390488,...,10.000000,1,0,0,0,0,0,0,0,1
4688,0,{},-0.153007,-0.102148,-1.563089,-0.929348,3.273799,Deceased,1,-0.337606,...,10.000000,1,0,0,0,0,0,0,0,1


In [42]:
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR_power2',
    'DonorAge_NOTR',
    'DonorAge_NOTR_power2',
    'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
    'Donor_Type', 
    'DialysisYears',

]
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
df_weight = find_ipw(df, confounders, treatments, scaler=None, verbose=False)

cph = CoxPHFitter()
cols = [
    # 'No_DESA',
    'Specific_DESA',
    'Other_DESA',
    'E', 'T', 
    'w'
]
cph.fit(
    df_weight[cols], 
    weights_col='w',
    duration_col='T', event_col='E', robust=True)
print(cph.print_summary())

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 14035.4 total observations, 9843.98 right-censored observations>
             duration col = 'T'
                event col = 'E'
              weights col = 'w'
          robust variance = True
      baseline estimation = breslow
   number of observations = 14035.4
number of events observed = 4191.42
   partial log-likelihood = -38510.38
         time fit was run = 2022-06-28 12:17:43 UTC

---
                coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                             
Specific_DESA   1.03       2.80       0.12             0.80             1.26                 2.22                 3.53
Other_DESA      0.15       1.16       0.14            -0.13             0.43                 0.88                 1.53

                cmp to    z      p   -log2(p)
covariate                                    
Specific_DESA     0.00 8.72 <0.005      58.28
Other_DESA        0.00 1.06   0.29       1.80
---
Concordance = 0.54
Partial AIC = 77024.76
log-likelihood ratio test = 932.44 on 2 df
-log2(p) of ll-ratio test = 672.62

None


In [24]:
df.Relevant_DESA_Bad.value_counts()

0    4494
1     196
Name: Relevant_DESA_Bad, dtype: int64